In [ ]:
using Knet, Plots, Images

# Linear regression example with housing data

In [ ]:
# Download the housing dataset from the UCI Machine Learning Repository
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
file=Knet.dir("data","housing.data")
download(url,file)

In [ ]:
data = readdlm(file)'  # read blank separated data

In [ ]:
y = data[14:14,:]  # output

In [ ]:
x = data[1:13,:]   # input

In [ ]:
x = (x .- mean(x,2)) ./ std(x,2) # Data normalization

In [ ]:
predict(w,x)=(w[1]*x.+w[2])

In [ ]:
loss(w,x,y)=(sum(abs2,y-predict(w,x)) / size(x,2))

In [ ]:
lossgradient = grad(loss)

In [ ]:
srand(42)
w = [ 0.1*rand(1,13), 0.0 ]  # Initialize model

In [ ]:
loss(w,x,y)

In [ ]:
lossgradient(w,x,y)

In [ ]:
w[2]=0.1

In [ ]:
loss(w,x,y)

In [ ]:
function train!(w, data; lr=.1)
    for (x,y) in data
        dw = lossgradient(w, x, y)
        for i in 1:length(w)
            w[i] -= lr * dw[i]
        end
    end
    return w
end

In [ ]:
weights = [ copy(train!(w, [(x, y)])) for epoch=1:10 ]

In [ ]:
losses = [ loss(w,x,y) for w in weights ]

In [ ]:
using Plots
plot(losses,xlabel="Epochs",ylabel="Loss") 

# Load and minibatch MNIST data

In [ ]:
load(Pkg.dir("Knet/docs/src/images/firsteightimages.jpg"))

In [ ]:
include(Knet.dir("examples","mnist.jl"))
MNIST.loaddata()
using MNIST: xtrn,ytrn,xtst,ytst,minibatch
Atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32}
dtst = minibatch(xtst,ytst,100;atype=Atype); # [ (x1,y1), (x2,y2), ... ] where xi,yi are minibatches of 100
dtrn = minibatch(xtrn,ytrn,100;atype=Atype); # [ (x1,y1), (x2,y2), ... ] where xi,yi are minibatches of 100

In [ ]:
length(dtrn),length(dtst)

In [ ]:
(x,y) = dtst[1];

In [ ]:
display(x)

In [ ]:
display(y)

# Softmax classification example with MNIST

## Define softmax loss

In [ ]:
# Average loss for a single (x,y) minibatch
function loss(w,x,ygold)
    ypred = predict(w,x)
    ynorm = ypred .- log.(sum(exp.(ypred),1))
    -sum(ygold .* ynorm) / size(ygold,2)
end

# Average loss for a dataset of (x,y) pairs
avgloss(w,data)=mean(loss(w,x,y) for (x,y) in data)

In [ ]:
srand(42)
wsoft=map(Atype, [ 0.1*randn(10,784), zeros(10,1) ])
loss(wsoft, x, y)

In [ ]:
avgloss(wsoft, dtst)

## Train softmax model

In [ ]:
softmodels = [ copy(train!(wsoft, dtrn)) for epoch=1:100 ];

## Plot softmax learning curve

In [ ]:
trnsoftloss = [ avgloss(w,dtrn) for w in softmodels ];
tstsoftloss = [ avgloss(w,dtst) for w in softmodels ];

In [ ]:
plot([trnsoftloss tstsoftloss],ylim=(.2,.36),labels=[:trnsoftloss :tstsoftloss],xlabel="Epochs",ylabel="Loss") 


## Plot softmax error rate

In [ ]:
# Average error for a single (x,y) minibatch
function err(w,x,ygold)
    ypred = predict(w,x)
    ncorr = sum(ygold .* (ypred .== maximum(ypred,1)))
    return 1 - ncorr/size(x,2)
end

# Average error for a dataset of (x,y) pairs
avgerr(w,data)=mean(err(w,x,y) for (x,y) in data)

In [ ]:
trnsofterr = [ avgerr(w,dtrn) for w in softmodels ];
tstsofterr = [ avgerr(w,dtst) for w in softmodels ];

In [ ]:
plot([trnsofterr tstsofterr],ylim=(.06,.10),labels=[:trnsofterr :tstsofterr],xlabel="Epochs",ylabel="Error")

# Multilayer perceptron example with MNIST

In [ ]:
# We only need to change the predict function!
function predict(w,x)
    for i=1:2:length(w)
        x = w[i]*x .+ w[i+1]
        if i<length(w)-1
            x = max.(0,x)                         
        end
    end
    return x
end

In [ ]:
srand(42)
wmlp=map(Atype, [ 0.1*randn(64,784), zeros(64,1), 
                  0.1*randn(10,64),  zeros(10,1) ])
loss(wmlp, x, y)

## Train MLP model

In [ ]:
mlpmodels = [ copy(train!(wmlp, dtrn)) for epoch=1:100 ];

## Compare MLP loss with softmax loss

In [ ]:
trnmlploss = [ avgloss(w,dtrn) for w in mlpmodels ];
tstmlploss = [ avgloss(w,dtst) for w in mlpmodels ];

In [ ]:
plot([trnsoftloss tstsoftloss trnmlploss tstmlploss],ylim=(.0,.36),labels=[:trnsoftloss :tstsoftloss :trnmlploss :tstmlploss],xlabel="Epochs",ylabel="Loss") 

## Compare MLP error with softmax error

In [ ]:
trnmlperr = [ avgerr(w,dtrn) for w in mlpmodels ];
tstmlperr = [ avgerr(w,dtst) for w in mlpmodels ];

In [ ]:
plot([trnsofterr tstsofterr trnmlperr tstmlperr],ylim=(.0,.10),labels=[:trnsofterr :tstsofterr :trnmlperr :tstmlperr],xlabel="Epochs",ylabel="Error")

# CNN example with MNIST

In [ ]:
# We only need to change the predict function!
function predict(w,x,n=length(w)-4) # LeNet model
    x = reshape(x, (28,28,1,div(length(x),784)))
    for i=1:2:n
        x = pool(relu.(conv4(w[i],x) .+ w[i+1]))
    end
    for i=n+1:2:length(w)-2
        x = relu.(w[i]*mat(x) .+ w[i+1])
    end
    return w[end-1]*x .+ w[end]
end

In [ ]:
srand(42)
wcnn=map(Atype, [ 0.1*randn(5,5,1,20),  zeros(1,1,20,1), 
                  0.1*randn(5,5,20,50), zeros(1,1,50,1),
                  0.1*randn(500,800),  zeros(500,1),
                  0.1*randn(10,500),  zeros(10,1) ])
loss(wcnn, x, y)

## Train CNN model

In [ ]:
cnnmodels = [ copy(train!(wcnn, dtrn)) for epoch=1:100 ];

## Compare CNN loss with MLP

In [ ]:
trncnnloss = [ avgloss(w,dtrn) for w in cnnmodels ];
tstcnnloss = [ avgloss(w,dtst) for w in cnnmodels ];

In [ ]:
plot([trnsoftloss tstsoftloss trnmlploss tstmlploss trncnnloss tstcnnloss],ylim=(.0,.36),labels=[:trnsoftloss :tstsoftloss :trnmlploss :tstmlploss :trncnnloss :tstcnnloss],xlabel="Epochs",ylabel="Loss") 

## Compare CNN error with MLP

In [ ]:
trncnnerr = [ avgerr(w,dtrn) for w in cnnmodels ];
tstcnnerr = [ avgerr(w,dtst) for w in cnnmodels ];

In [ ]:
plot([trnsofterr tstsofterr trnmlperr tstmlperr trncnnerr tstcnnerr],ylim=(.0,.10),labels=[:trnsofterr :tstsofterr :trnmlperr :tstmlperr :trncnnerr :tstcnnerr],xlabel="Epochs",ylabel="Error")

# Shakespeare example

In [ ]:
txt = readstring(Knet.dir("data/100.txt"));

In [ ]:
length(txt)

In [ ]:
println(txt[1000000:1001000])

In [ ]:
include("charlm.jl")

In [ ]:
model = Knet.dir("data/shakespeare.jld")

In [ ]:
CharLM.main("--load $model --gen 1000");

# VGG example

In [ ]:
include("vgg.jl");

In [ ]:
using FileIO, Images

In [ ]:
img = load(Knet.dir("data/cat.jpg"))

In [ ]:
VGG.main(Knet.dir("data/cat.jpg"))

In [ ]:
VGG.main("https://cvimg1.cardekho.com/p/237x156/in/mahindra/torro-25/mahindra-torro-25.jpg")

In [ ]:
1+2